In [ ]:
import os
import subprocess

In [ ]:
! apt-get install ffmpeg

In [ ]:
! wget https://gif.ski/gifski-0.10.1.zip
! unzip gifski-0.10.1.zip

In [ ]:
os.chdir('linux')
! dpkg -i gifski_0.10.1_amd64.deb

In [ ]:
os.chdir('/content')
! git init
! git remote add -f origin https://github.com/fengredrum/tiny-gif-converter.git
! git config core.sparsecheckout true
! echo sample >> .git/info/sparse-checkout
! git pull origin master

In [ ]:
def gif_converter(
    load_path=None,
    save_path=None,
    start_time='00:00:00',
    duration='5.',
    fps='10',
    frame_width='320',
):

    # Setup path
    if load_path is None:
        load_path = 'sample.mp4'
    if save_path is None:
        save_path = 'output/'
        os.makedirs(save_path, exist_ok=True)

    subprocess.call(['mkdir', 'tmp/'])

    # Extract frames
    subprocess.call([
                    'ffmpeg',
                    '-ss', start_time,  # start time
                    '-t', duration,  # duration
                    '-i', load_path,  # original file dir
                    '-vf', "fps=" + fps + ", scale=" + frame_width + ":-1",
                    '-pix_fmt', 'rgb24',  # pixel formats
                    # output file dir
                    '-y', 'tmp/frame%04d.png',
                    ])

    # Create GIF
    out_file_name = os.path.basename(load_path) + '_converted.gif'
    ex = subprocess.Popen("gifski --fps " + fps + " -o " +
                          os.path.join(save_path, out_file_name) +
                          " tmp/frame*.png", 
                          stdout=subprocess.PIPE,
                          universal_newlines=True,
                          shell=True)

    while ex.poll() == None:
        # yield ex.stdout.readline()
        print(ex.stdout.readline())
    status = ex.wait()
    subprocess.call(['rm', '-rf', 'tmp/'])

In [ ]:
load_path = 'sample/sample_2.mp4'
gif_converter(load_path=load_path)

In [ ]:
from IPython.display import Image
out_path = os.path.join('output', load_path + '_converted.gif')
Image(open(out_path,'rb').read())